In [2]:
from dotenv import load_dotenv
import os
from google.cloud import bigquery
import google.generativeai as genai
from IPython.display import Markdown
import textwrap


/home/raihan12/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# SET UP

In [4]:
load_dotenv()

True

In [5]:
GOOGLE_APPLICATION_CREDENTIALS = os.environ["GOOGLE_APPLICATION_CREDENTIALS"]
client = bigquery.Client()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
model = genai.GenerativeModel('gemini-pro')

In [6]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [10]:
#Write Query on BQ
QUERY = """
SELECT id, description FROM `capstone-alterra-424814.dim_tables.dim_complaints` WHERE TIMESTAMP_TRUNC(updated_at, DAY) >= TIMESTAMP("2024-04-17") LIMIT 1000
  """

Query_Results = client.query(QUERY)
df = Query_Results.to_dataframe()

/home/raihan12/.local/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


# DATA

In [11]:
df

,id,description
0,C-abc123def456,Fasilitas kesehatan perlu diperbanyak di daera...
1,C-qazw7edcrfvgt,Transportasi malam hari sering terlambat. Hal ...
2,C-poiuytrqazsx,Pendataan penduduk desa kami belum dilakukan. ...
3,C-5f4e3d2c1b,Lingkungan di sekitar kami tercemar limbah ind...
4,C-qazwsxedc0r,Ketersediaan dokter di puskesmas perlu ditingk...
...,...,...
61,C-edcr37gt9hb,Transportasi malam hari sering terlambat
62,C-qwertyuiop1,Infrastruktur jalan rusak parah setelah musim ...
63,C-poiuyt11xx,Keamanan di perumahan kami sering terjadi penc...
64,C-4c6baaeff8,tolong min lauknya ditambah


In [ ]:
def get_complaint_by_id(id_complaint, dataset):
    try:
        complaint = dataset.loc[dataset['id'] == id_complaint, 'description'].values[0]
        return complaint
    except IndexError:
        return None

def recommendation(id_complaint, dataset):
    complaint = get_complaint_by_id(id_complaint, dataset)
    if not complaint:
        return "Complaint ID not found in the dataset."
    
    prompt = f"""
                Anda adalah seorang admin yang bertugas mengelola keluhan dan komplain di masyarakat wilayah provinsi di Banten, Suatu hari ada komplain yang masuk seperti berikut
                {complaint}
                berdasarkan komplain yang diadukan bagaimana anda akan menjawab keluhan tersebut? (bukan berupa surat formal, hanya saran jawaban)
            """
    response = model.generate_content(prompt)

    return response

id = input("Masukkan ID Complaint : ")

hasil = recommendation(id, df)

# Assuming 'text' property exists within GenerateContentResponse
response_text = hasil.text
print("\n-----------------------")
print("Hasil Analisis:")
print("\n-----------------------")
print(response_text)